In [ ]:
import numpy as np
import pandas as pd
from math import factorial as fact
from copy import deepcopy
import time

# 1. Inference

### 1.0. Solver

In [88]:
def Solver(state, size):
    
    print("----------------Initial State----------------")
    print(state)
    print("---------------------------------------------")

    m, n = size
    global move_count
    move_count=0
    if not IsSolvable(state, size):
        print("Given puzzle is unsolvable")
        return

    print("Make State Transition Table...")
    STT,hash_array,STT2,hash_array2 = MakeSTT()    
    
    # blank_tile 추적 
    blank_loc=GetLocation(state, 0)
    
    # first (m-2) rows
    for row in range(m-2):
        
        # first (n-2) columns
        for col in range(n-2):
            target_number = row*n + col + 1 
            target_loc = (row, col)
            
            # 이미 배치가 잘 되어 있으면 생략 
            if state[target_loc] == target_number:
                continue
            
            blank_loc = DirectMove(state, size, blank_loc, target_number, target_loc, global_row=row, global_col=col)

        # last 2 columns
        # t1,t2를 3x2  puzzle 안에 최단거리로 할당 
        t1 =row*n + n-1
        t2 =  t1+1
        loc1, loc2 = GetLocation(state, t1), GetLocation(state, t2) 
        
        # 이미 5,6 column의 배치가 잘 되어있을 시 생략 
        if (loc1==(row, n-2)) and (loc2==(row, n-1)):
            continue 

        target_loc1, target_loc2 =GetShortestDouble(state, size,row,t1, t2, loc1, loc2)

        # 우측 상단에 있는 타일 먼저.
        blank_loc = SelectMovingOrder_3x2(state, size, blank_loc, t1, t2, loc1, loc2, row, target_loc1, target_loc2)
        
        # blank tile 할당 
        blank_path = CheckBlankPath(state, size, blank_loc, row, target_loc1, target_loc2)
        for blank_direct in blank_path:
            blank_loc = move(state, size, blank_loc, blank_direct)
        
        # Run STT
        sub_puzzle = state[row:row+3, n-2:] # 3x2 sub-puzzle
        reg_sub_puzzle = MatchToStandard(sub_puzzle, (3,2), t1, t2)
        
        
        STT_path = GetShortestPath(STT, hash_array,reg_sub_puzzle)
        for idx, blank_direct in enumerate(STT_path):
            # t1, t2만 맞춰지면 Stop 
            if (state[row][n-2] ==t1) and (state[row][n-1]==t2):
                break
            
            blank_loc = move(state, size, blank_loc, blank_direct)


    # 마지막 2행.

    target_row = m-3 # 건드리면 안되는 행
    for col in range(n-3):
        t1= (target_row+1)*n+col +1 
        t2 = t1 + n
        
        loc1 = GetLocation(state, t1)
        loc2 = GetLocation(state, t2) 
        
        # 이미 5,6 column의 배치가 잘 되어있을 시 다음 column으로 pass
        if (loc1==(target_row+1, col)) and (loc2==(target_row+2, col)):
            continue 
        
        # t1,t2의 최단거리 할
        target_loc1, target_loc2 =GetShortestDouble2(state, size,col,t1, t2, loc1, loc2)
        
        # 좌측 상단의 타일부터 이
        blank_loc = SelectMovingOrder_2x3(state, size, blank_loc, t1, t2, loc1, loc2, target_loc1, target_loc2)
        
        

        # blank_tile 할당 
        blank_path = CheckBlankPath2(state, size, blank_loc, col, target_loc1, target_loc2)

        # blank_tile 이
        for blank_direct in blank_path:
            blank_loc = move(state, size, blank_loc, blank_direct)
        

        # run STT
        sub_puzzle = state[m-2:, col:col+3] # 3x2 sub-puzzle
        reg_sub_puzzle = MatchToStandard(sub_puzzle, (2,3), t1, t2)
        STT_path = GetShortestPath(STT2, hash_array2,reg_sub_puzzle)
        
        for idx, blank_direct in enumerate(STT_path):
            # t1, t2만 맞춰지면 Stop
            if (state[m-2][col] ==t1) and (state[m-1][col]==t2):
                break
            
            blank_loc = move(state, size, blank_loc, blank_direct)
        

    # 마지막 2x3 Subpuzzle
    # 이미 모든 숫자가 다 들어가있다.

    sub_puzzle = state[m-2:, n-3:]
    t1 = (m-2)*(n)+4
    t2 = t1+6

    # run STT
    reg_sub_puzzle = MatchToStandard(sub_puzzle, (2,3), 22, 28, last=True)
    STT_path = GetShortestPath(STT2, hash_array2,reg_sub_puzzle)

    for idx, blank_direct in enumerate(STT_path):
        blank_loc = move(state, size, blank_loc, blank_direct)
    

    print("----------------Final State----------------")
    print(state)

    print("\n- Moves: ", move_count)

    print("-------------------------------------------")





### 1.1. Sample 1개
- Solvable : 약 0.065sec  
- Unsolvable : 약 0.001sec

In [87]:
random_state = np.random.permutation(np.arange(0,30)).reshape(5,6)

start = time.time()
Solver(random_state, size=(5,6))
end = time.time()
print("Run Time : {:.5f}sec".format(end-start))


----------------Initial State----------------
[[ 5  8  9 23 13  1]
 [11 22 20 27  2  6]
 [12 19  4  7 25 17]
 [10 28 18 29  3 14]
 [26 21 16 24 15  0]]
---------------------------------------------
Make State Transition Table...
----------------Final State----------------
[[ 1  2  3  4  5  6]
 [ 7  8  9 10 11 12]
 [13 14 15 16 17 18]
 [19 20 21 22 23 24]
 [25 26 27 28 29  0]]

- Moves:  358
-------------------------------------------
Run Time : 0.06563sec


### 1.2. Sample 100개(2.87sec, 출력 제외)

In [65]:
start = time.time()
for i in range(100):
    random_state = np.random.permutation(np.arange(0,30)).reshape(5,6)
    Solver(random_state, size=(5,6))
end = time.time()
print("Run Time : {:.5f}sec".format(end-start))


Run Time : 2.87867sec


# 2. Function

In [23]:
# Bidirectional BFS  / State Transition Table을 만드는 용도.
def BBFS(target_state, size):
    hash_array=np.zeros(1000) # hash_array[hash_value] = state sequence
    order = {0: "up", 1:"right", 2:"down", 3:"left"}
    m,n = size # (3,2) 혹은 (2,3)

    hash_value = hash(target_state)

    next_seq=1
    prev_seq = -1
    reverse_direct = "out"
    hash_array[hash_value]=next_seq # state 방문 처리
    
    queue=[]

    # 현재 state, blank 위치, sequence,  hash value, previous sequence )
    queue.append((target_state, (m-1, n-1), next_seq, hash_value, prev_seq, reverse_direct))
    # c에선 구조체로 바꿔주자.

    full_seq=[]
    full_hash=[]
    full_prev_seq=[]
    full_reverse_direct=[]
    while queue:

        current = queue.pop(0)

        prev_state = current[0] # 일단 구조체가 아니라 풀어서 씀.
        prev_blank = current[1]
        prev_seq = current[2]
        prev_hash = current[3]  
        prev_prev_seq = current[4]
        prev_reverse_direc = current[5]

        full_seq.append(prev_seq)
        full_hash.append(prev_hash)
        full_prev_seq.append(prev_prev_seq)
        full_reverse_direct.append(prev_reverse_direc)
        
        for key, direct in order.items():
            reverse_direct = order[(key+2)%4] # 백트래킹용 역방향
            next_state_and_loc  = move_sst(prev_state, size, prev_blank, direct)

            if next_state_and_loc: # 움직일 수 있으면
                next_hash = hash(next_state_and_loc[0]) # hash값 추출
                if not hash_array[next_hash]: # 아직 방문되지 않았으면, 
                    
                    next_seq = next_seq + 1
                    queue.append((next_state_and_loc[0], next_state_and_loc[1], next_seq, next_hash, prev_seq, reverse_direct)) # 큐에 삽입
            
                    hash_array[next_hash] = next_seq # 방문처리 
                else:
                    pass
                    

    STT=pd.DataFrame({"seq":full_seq, "hash":full_hash, "prev_seq":full_prev_seq, "reverse_direct": full_reverse_direct})
    return STT, hash_array


# STT를 이용해 3x2 Sub-puzzle을 최단경로로 풀어주는 함수      
def GetShortestPath(STT, hash_array, initial_state):
    
    hash_value = hash(initial_state)
    
    next_seq=int(hash_array[hash_value]) # initial state
    prev_seq = STT.iloc[next_seq-1]["prev_seq"] # next seq -> prev seq ()

    full_direct=[]
    while (prev_seq!=-1): # 각 행의 마지막 2개 타일 정렬이 끝날 때까지
        next_direct = STT.iloc[next_seq-1]["reverse_direct"] # 더 가야 한다.
        # print(next_seq, next_direct)
        full_direct.append(next_direct) # 방향 저장
        
        next_seq = prev_seq
        prev_seq = STT.iloc[next_seq-1]["prev_seq"]

    # 최적화 더 해야함. 굳이 끝까지 맞출 필요 없음.(함수 더 짤 때 ㄱㄱ)
    
    return full_direct

# 3x2 puzzle을 풀기위해 [1,2,3,4,5,0] 형태로 정규화가 필요함.
def MatchToStandard(initial_state, size, t1, t2, last=False): # last : Last 2x3 Puzzle
    # 단, solvable하게.

    m, n = size  
    standard_initial_state = np.zeros((m, n))


    # 임의의 initial state에서 STT 돌리려면 
    # 1,2,0을 매칭시키고 나머지를 3,4,5 넣을 때 적절히 잘 넣어야 한다. 

    if (m,n)==(3,2):
        others = [3,4,5] # for solvable
        
    else: # 아마 2x3 퍼즐만.
        if not last:
            others = [2,3,5] # Solvable
        else: # last
            pass
        
    idx=0        
    for i in range(m):
        for j in range(n):
            if (initial_state[i,j] == t1):
                standard_initial_state[i,j] = 1
            elif (initial_state[i,j]==t2):
                if (m,n)==(3,2):
                    standard_initial_state[i,j] = 2
                else: 
                    standard_initial_state[i,j] = 4
                    
            elif (initial_state[i,j]==0):
                standard_initial_state[i,j] = 0
            else: # 나머진 대충
                if not last:
                    standard_initial_state[i,j] = others[idx]
                    idx+=1
                if last: # 정해진 매칭(자동으로 solvable)
                    if (initial_state[i,j]==23):
                        standard_initial_state[i,j] = 2
                    elif (initial_state[i,j]==24):
                        standard_initial_state[i,j] = 3
                    else: # 29
                        standard_initial_state[i,j] = 5

    if (not IsSolvable(standard_initial_state, size)) and (not last):
        # print("Not Solvable", standard_initial_state)
        loc2 = np.where(standard_initial_state==others[0])[0][0], np.where(standard_initial_state==others[0])[1][0]
        loc5 = np.where(standard_initial_state==others[2])[0][0], np.where(standard_initial_state==others[2])[1][0]

        # [3,4,5](3x2), [2,3,5](2x3) 에서 각각 3<->5 5<->3 숫자 변경

        standard_initial_state[loc2], standard_initial_state[loc5] = standard_initial_state[loc5], standard_initial_state[loc2]
        # print("Solvable", standard_initial_state)
        if not IsSolvable(standard_initial_state, size):
            raise NameError("MatchToStandard에서의 오류")   
    
    return standard_initial_state
    


# Number [target_number]를 위치 target_loc으로 이동하는 함수
def DirectMove(state, size, blank_loc, target_number, target_loc, global_row=2, global_col=5, forward=True, fbd_loc=False, fbd_loc2=False):
    # initial loc
    number_loc = GetLocation(state, target_number)

    if number_loc == target_loc: # 움직일 필요 없을 때
        return blank_loc

    # 좌, 우를 맞추고 위로 가자.
    lr = target_loc[1] - number_loc[1] # -4면 왼쪽으로 4칸, +2면 오른쪽으로 2칸
    up = target_loc[0] - number_loc[0] # 0이면 그대로, +3이면 위로 3칸, 음수는 나올 일 없(는줄 알았는데 있다. )


    # 사실 나중에는 한 번에 처리할 수 있지만, 일단 가독성을 위해 펼쳐놓음 
    if lr<0:
        lr *= -1 # 그냥 편의상 양수로.
        blank_loc_to_number = "left" # 숫자를 왼쪽으로 가려면 blank tile을 숫자 왼쪽으로 
        blank_direct_to_number = "right" # 숫자가 왼쪽으로 가려면 blank tile을 오른쪽으
    elif lr>0:
        blank_loc_to_number = "right" 
        blank_direct_to_number = "left"

    # 좌,우 이동
    for j in range(lr):
        
        blank_path = GetBlankPath(size, blank_loc, number_loc, blank_loc_to_number, target_loc[0], target_loc[1], forward, global_row, global_col ,fbd_loc, fbd_loc2)
        
        for blank_direct in blank_path:
            # print("이동 : ", blank_direct)
            blank_loc = move(state, size, blank_loc, blank_direct)
        
        number_loc = blank_loc # 숫자가 옮겨질 위치는 move 이전의 blank tilepr
        blank_loc = move(state, size, blank_loc, blank_direct_to_number)

    
    # 위로 이동
    if up<0:
        for i in range(abs(up)):
            blank_path = GetBlankPath(size, blank_loc, number_loc, "up", target_loc[0], target_loc[1], forward, global_row, global_col ,fbd_loc, fbd_loc2)
            for blank_direct in blank_path:
                # print("이동 : ", blank_direct)
                blank_loc = move(state, size, blank_loc, blank_direct)
            
            number_loc = blank_loc
        
            
            blank_loc = move(state, size, blank_loc, "down")
            
    else: # 때로는 아래로 보내야 될 수도 있다. 그래도 낭비를 줄이기 위해 이전단계에서 철저하게 없애주자.  
        pass

    return blank_loc


In [28]:
def GetBlankPath(size, blank_loc, number_loc, number_direct,target_row, target_col=False, forward=True, global_row=2, global_col=5, fbd_loc = False, fbd_loc2=False):
    # number_direct: 즉, 각각 숫자가 가려는 위치와 같음)
    # forward:  앞의 1,2행 처리할 때 Rule
    # target_row : 위치하려는 높이
    # global_row, col : 배치 완료된 행과 열 
    # fbd_loc : 마지막 두 열 동시에 처리할 때, 처음 이동했던 타일은 안 건드리게끔 예외처리(전체적으로 이런 상황이 벌어지지 않게끔 프로그래밍 했으나, 극히 일부 샘플에서 문제를 보임).
    # fbd_loc2 : 마지막 두 행 처리시 어느 다른 하나를 역방향으로 움직이지 않게끔 우회 ("left" 시에만 신경쓰면 된다) 

    m,n=size

    lr = number_loc[1] - blank_loc[1] # e.g., -2이면 blank가 2칸 오른쪽에 있다. +1이면 blank가 1칸 왼쪽에 있다. 
    ud = number_loc[0] - blank_loc[0] # e.g., -2이면 blank를 2칸 아래쪽에 있다. +1이면 blank가 1칸 위쪽에 있다. 
    path=[]
    if number_direct == "left":
        lr -= 1 # Block tile이 숫자의 좌측에 위치해야 
    elif number_direct =="right":
        lr += 1 # Block tile이 숫자의 우측에 위치해야 
    elif number_direct =="up":
        ud -=1 # Block tile이 숫자의 위쪽으로 위치해야 (위가 음수임)
    
    # 일단 "left"&"right"에서,  
    if (number_direct == "left") or (number_direct =="right"):
        # 마지막 두 행에서, 두 타일 모두 최단거리를 보전해주기 위함. 
        if fbd_loc2: # 마지막 두 행 처리할 때, [[19, *, 0], [*, *, 25]] 꼴이라면 25를 건들지 않는 선에서 0을 19의 왼쪽으로 옮겨주어야 한다.
            
            if  (fbd_loc2[1]>=(number_loc[1]+1)): # 25가 19의 우측에 존재할 때 
                if blank_loc[1]>=fbd_loc2[1]: # 공백이 25를 지나칠 가능성이 있을 경우에만 
                    if (number_loc[0]==(m-2)) and (fbd_loc2[0]==(m-1)): # 19가 위, 25가 아래에 있다면 (아마 blank path도 거의 위에 있을 것..)
                        path = path+ ["left"] * (abs(lr)-2) # 19의 오른쪽까지 미리 이동해준다.
                        lr = lr + (abs(lr)-2) # 가야될 숫자 보정.
                    elif (number_loc[0]==(m-1)) and (fbd_loc2[0]==(m-2)) : # 25가 위, 19가 아래에 있다면
                        path = path + ["down"]+["left"]*(abs(lr)-2) # 25가 역방향으로 가지 않게 하기 위해 우회를 해줄 필요가 있다. 
                        lr = lr + (abs(lr)-2) # 
                        ud -=1 # 보정
                        blank_loc = (blank_loc[0]+1, blank_loc[1]-(abs(lr)-2)) # 얘만 반영 해주어야 한다. 사전 우회임.
                    else: # 두 숫자의 행이 같을 땐 신경 안 써줘도 된다. 
                        pass

        # blank tile과 number tile이 같은 행에 있고(ud==0이고),
        if number_loc[0] == blank_loc[0]:
            # blank tile이 number tile을 거슬러가야 한다면      
            if ( ( (number_direct == "left") and (lr<0) ) or ( (number_direct =="right") and (lr>0) ) ):  
                if (blank_loc[0] >(global_row+1)) : # 그리고 이미 맞춰진 행을 건들지 않는 선에서   
                    if (number_loc[0])==(m-1): # 마지막 행이라면 무조건 위로만 이동. 
                        path.append("up")
                        ud+=1 
                    else: 
                        if fbd_loc:
                            if (fbd_loc[0]==blank_loc[0]-1): # 위로 갈 수 없는 경우 가면 큰일나는 경우
                                path.append("down") # 아래로 피해준다. 
                                ud-=1
                            else:
                                path.append("up") # 나머진 위로 피해도 무방.
                                ud+=1 # 
                        else: # 마지막 행이 아니라면 너 마음대로 해라.
                            path.append("up") 
                            ud+=1 
                else: # 맞춰진 행을 건들 위험성이 있다면, 무조건 아래쪽으로 가야 한다. 
                    path.append("down") # 단, 이미 맞춰진 행을 안 건드려
                    ud-=1 # 역시 마지막에 위로 한 번에 더 이동해주어야 함.
    
        if fbd_loc:
             # blank_tile이 바로 오른쪽으로 못 가고, 움직일 타일이 아래나 오른쪽에 있다면(즉, blank_tile이 오른쪽으로 가야한다면)
            if (blank_loc[0]==fbd_loc[0]) and ((blank_loc[1]+1)==fbd_loc[1]) and (blank_loc[0]<number_loc[0]) and (blank_loc[1]<=number_loc[1]):
                if ((blank_loc[0]+1)==number_loc[0] and ((blank_loc[1]==number_loc[1]))): # 특히, 움직일 타일이 바로 아래쪽에 있다면
                    path = path + ["left", "down", "down"]
                    lr+=1
                    ud-=2
                
                elif ((blank_loc[0]+1)==number_loc[0]): # 아니더라도, 바로 아래에 있으면 두 번 우회해야.
                    path = path + ["down", "down"]
                    ud-=2 
                
                else: # 그 외의 경우에는, 알고리즘 시스템 상, 아래로만 미리 한 번 가주면 된다.
                    path.append("down")
                    ud-=1

            # blank_tile이 2칸 이상은 오른쪽으로 못 갈 경우. 
            elif (blank_loc[0]==fbd_loc[0]) and ((blank_loc[1]+2)<=fbd_loc[1]) and (blank_loc[0]<number_loc[0]) and (blank_loc[1]<number_loc[1]):
                
                # 우측 하단에 움직일 타일이 있는 경우 
                if ((blank_loc[0]+1)==number_loc[0]) and (blank_loc[1]+1==number_loc[1]) and (blank_loc[1]+2==fbd_loc[1]):
                    path = path + ["down", "down"]
                    ud-=2
                
                elif number_loc[1]>=(fbd_loc[1]-1): # fbd를 침범할 가능성이 있는 경우.
                    path.append("down")
                    ud-=1
                    
        # 이제 blank tile과 number tile은 같은 곳에 있지 않다. 좌우로 이동해주면 된다. 
        
        ## 단, blank tile이 맞춰진 행을 건들지 않는 선에서.   
        # 일단, 초반엔 맞춰진 열을 건드리는 경우를 방지하기 위해 처리
        if global_row<m-2:
            if (blank_loc[1]==(global_col)) and (blank_loc[0]==global_row):
                path.append("down")
                ud-=1 
        if lr<0:
            for i in range(abs(lr)):
                path.append("left")
        elif lr>0:
            for i in range(lr):
                path.append("right")
        
        # 마지막으로 상,하를 맞춰주면 된다.
        if ud<0:
            for i in range(abs(ud)):
                path.append("up")
        elif ud>0:
            for i in range(ud):
                path.append("down")

    # 위로 가야한다면?
    if (number_direct == "up"):
        # blank tile과 number tile이 같은 열에 있고(lr==0이고),
        if number_loc[1] == blank_loc[1]: #(lr==0이)
            # blank tile이 number tile을 거슬러가야 한다면      
            if (ud<0): 
                if (blank_loc[1]==n-1) : # 단, 마지막 열이면
                    path.append("left") # 왼쪽으로 피해주고 
                    lr+=1 # 마지막에 오른쪽으로 이동 
                elif fbd_loc: # fbd_loc 존재할 때
                    if fbd_loc[1]==(blank_loc[1]+1) : # 오른쪽에 건드려선 안될 애가 있다면
                        path.append("left") # 왼쪽으로 우회하자.
                        lr+=1
                    else:    
                        path.append("right") # 오른쪽으로 피해주고 
                        lr-=1 # 역시 마지막에 왼쪽으로 이동해주어야 함.     
                else:
                        path.append("right") # 오른쪽으로 피해주고 
                        lr-=1 # 역시 마지막에 왼쪽으로 이동해주어야 함.     
                    
        # 이제 blank tile과 number tile은 같은 곳에 있지 않다. 위로만 움직여주자.

        # 바로 위에 정리된 타겟이 있다면?
        if forward and (number_loc[0]==target_row+1) and blank_loc[1]<=(target_col-1): 
            if blank_loc[0]==number_loc[0]: # 행이 같다면? 
                path.append("down") # 우회필요
                ud-=1 # 위로 다시 가주어야 함. 

            temp_gap= target_col - blank_loc[1] +1 # 
            path = path + ["right"]*temp_gap # 보정 
            lr-=temp_gap # 역시 왼쪽으로 다시 가주어야 함.

        if ud<0:
            for i in range(abs(ud)):
                path.append("up")
        elif ud>0:
            for i in range(ud):
                path.append("down")
        
        # 마지막으로 좌,우를 맞춰주면 된다.
        # lr이 -3이면 왼쪽으로 3칸 이동, lr이 +2면 오른쪽으로 2칸 이동
        if lr<0:
            for i in range(abs(lr)):
                path.append("left")
        elif lr>0:
            for i in range(lr):
                path.append("right")

    return path


def GetShortestDouble(state, size, target_row, t1, t2,loc1,loc2):
    m,n = size
    target_loc1 = (-1, -1)
    target_loc2 = (-1, -1)
    sub_puzzle=state[target_row: target_row+3, n-2:]

    # 이미 퍼즐 안에 t1,t2이 들어와 있다면
    if (t1 in sub_puzzle)and(t2 in sub_puzzle): # 3x2 Puzzle
        if target_row == (m-3): # 마지막 수행할 때
            if (not (0 in sub_puzzle)) and (loc1[1]==n-2) and (loc2[1]==n-2) and (loc1[0]>m-3) and (loc2[0]>m-3): # blank가 포함 안 되고, 앞 길을 다 막는다면 
                if loc1[0]<loc2[0]: # 위에 있는 친구를
                    target_loc1 = (loc1[0],n-1) # 끝으로 미뤄준다.
                    target_loc2 = loc2
                else: 
                    target_loc2 = (loc2[0], n-1) 
                    target_loc1 = loc1 
            else:
                target_loc1 = loc1
                target_loc2 = loc2
        else:
            target_loc1=loc1
            target_loc2=loc2

    # 두 숫자가 모두 안 들어와있을 때
    elif not ( (t1 in sub_puzzle) or (t2 in sub_puzzle) ):
        # 같은 행에 있으면
        if loc1[0]==loc2[0]:
            # SubPuzzle에 해당하는 행에 없다면
            if loc1[0]>target_row+2:
                target_loc_row = target_row+2 # Sub puzzle 내 가장 아래로. 
            else: # 있다면 상하로는 움직이지 않는다. 
                target_loc_row = loc1[0] # 

            if loc1[1]<loc2[1]: 
                target_loc2=(target_loc_row-1, n-1) # 오른쪽에 있는 타일은 맨 끝으로 (단, 서로 건드리지 않게 하기 위해 단차를 주자.)
                target_loc1=(target_loc_row, n-2) # 나머지 하나는 그 왼쪽으로
            else:
                target_loc1=(target_loc_row-1, n-1) # 오른쪽에 있는 타일을 맨 끝으로
                target_loc2=(target_loc_row,n-2) # 나머지 하나는 그 왼쪽으로

        else: # 다른 행에 있으면
            # t1이 더 위에 있을 때 
            if loc1[0]<loc2[0]:
                if loc1[0]>(target_row+2): # 높이있는 타일도 SubPuzzle 아래에 있을 때  
                    target_loc1_row = target_row+1
                    target_loc2_row = target_loc1_row + 1 # 그 아래
                elif loc1[0]<=(target_row+2) and loc2[0]>(target_row+2):
                    target_loc1_row = target_row+1  # 2행
                    target_loc2_row = target_loc1_row +1 # 그 아래에
                else: # 두 타일 모두 SubPuzzle과 같은 높이에 있을 때에 (아마 마지막은 항상 여기))
                    if (loc2[1]>=loc1[1]):
                        if (loc2[1]-1 == loc1[1]): # t2를 옮길 때 t1의 행이 안 바뀐다.
                            target_loc1_row = loc1[0]
                            target_loc2_row = loc2[0]
                        else: # 나머지는 t2의 행이 바뀐다.
                            target_loc1_row = loc1[0]
                            target_loc2_row = target_loc1_row # t1을 바꾸다보면 t2도 행이 바뀌므로.
                    else:
                        blank_loc = GetLocation(state, 0)
                        if (blank_loc[0]==loc1[0]) and (blank_loc[1]==loc2[1]): # blank_tile 아래에 t1가 있을 때 
                            target_loc1_row = loc1[0]
                            target_loc2_row = loc1[0] # 행 역전 
                        else:
                            target_loc1_row = loc1[0]
                            target_loc2_row = loc2[0]

            elif loc2[0]<loc1[0]: # t2이 더 위에 있을 때
                if loc2[0]>(target_row+2): # 높이있는 타일도 SubPuzzle 아래에 있을 때  
                    target_loc2_row = target_row+1
                    target_loc1_row = target_loc2_row + 1 # 그 아
                elif loc2[0]<=(target_row+2) and loc1[0]>(target_row+2):
                    target_loc2_row = target_row+1 # 2행에 위치시켜야.
                    target_loc1_row = target_loc2_row +1 # 그 아래에
                else: # 두 타일 모두 SubPuzzle과 같은 높이에 있을 때
                    if (loc1[1]>=loc2[1]): # 그리고 t1이 더 오른쪽에 있을 때. 
                        if (loc1[1]-1 == loc2[1]): # t2를 옮길 때 t1의 행이 안 바뀌는 경우
                            target_loc2_row = loc2[0]
                            target_loc1_row = loc1[0]
                        else: # 나머지는 t1의 행이 바뀐다.
                            target_loc2_row = loc2[0]
                            target_loc1_row = target_loc2_row # 같다.
                    else: # 위에 있는 t2가 더 오른쪽에 있을 때
                        blank_loc = GetLocation(state, 0)
                        if (blank_loc[0]==loc2[0]) and (blank_loc[1]==loc1[1]): # blank_tile 아래에 t1이 있을 때 
                            target_loc2_row = loc2[0]
                            target_loc1_row = loc2[0] # 행 역전 
                        else:
                            target_loc2_row = loc2[0]
                            target_loc1_row = loc1[0]         
            else: 
                pass
                
            # 혹시 마지막 퍼즐이라면
            if target_row==m-3: 
                if loc1[1]<loc2[1]: # t2이 더 오른쪽에 있다면
                    target_loc2 = (target_loc2_row, n-1)
                    target_loc1 = (target_loc1_row, n-2)
                elif loc1[1]>loc2[1]: # t1이 더 오른쪽에 있다면 
                    target_loc1 = (target_loc1_row, n-1)
                    target_loc2 = (target_loc2_row, n-2)
                else: # 둘이 같은 수직선상에 있다면, 그냥 위에 있는 친구를 오른쪽으로 몰아준다.
                    if loc1[0]<loc2[0]:
                        target_loc1 = (target_loc1_row, n-1)
                        target_loc2 = (target_loc2_row, n-2) # 이 때, target_loc1_row == target_loc2_row이다(어차피 t1를 이동하는 과정에서 t2가 위로 올라오게 된다. )
                    else:
                        target_loc2 = (target_loc2_row, n-1)
                        target_loc1 = (target_loc1_row, n-2) # 마찬가지로 target_loc2_row == target_loc1_row
                    
            else:
                if (loc1[0]<loc2[0]) and (target_loc1_row==target_loc2_row): # 이 때 target_loc1_row와 target_loc2_row는 같다.
                    target_loc1_col = n-1
                    target_loc2_col = n-2
                elif (loc2[0]<loc1[0]) and (target_loc1_row==target_loc2_row):
                    target_loc2_col = n-1
                    target_loc1_col = n-2
                else: # 나머지
                    # 열 배정
                    if loc1[1]>=n-2:
                        target_loc1_col = loc1[1]
                    else:
                        target_loc1_col = n-2
                    
                    if loc2[1]>=n-2:
                        target_loc2_col = loc2[1]
                    else:
                        target_loc2_col = n-2
                    
                target_loc1 = (target_loc1_row, target_loc1_col)
                target_loc2 = (target_loc2_row, target_loc2_col)

    else: # 두 숫자 중 하나만 퍼즐에 들어있을 때
        if t1 in sub_puzzle:
            target_loc1=loc1
            temp_in_loc = loc1
            temp_out_loc = loc2

        else:
            target_loc2=loc2
            temp_in_loc = loc2
            temp_out_loc = loc1

        if temp_out_loc[0]<=target_row+2: # out이 SubPuzzle과 같은 높이일 때
            if temp_in_loc[1]==(n-1): # in의 위치가 6열이라면
                temp_out_target = (temp_out_loc[0], n-2) # 5열에 위치
            else: # in의 위치가 5열이라면
                if temp_in_loc[0]!=temp_out_loc[0]: # 서로 행이 다르다면
                    blank_loc = GetLocation(state, 0)
                    if (temp_in_loc[0]!=(target_row))and (target_row == (m-3)): # 마지막 3x2 puzzle일 때는 길막 방지
                        if (temp_in_loc[0] == (m-2))  and (temp_out_loc[1] == blank_loc[1]) and (temp_out_loc[0]>blank_loc[0]): # 단, 블랭크 타일 아래에 out이 있다면 in을 밀다가 out을 끌어올려버린다.
                            temp_in_loc = (temp_in_loc[0], n-1) # in을 6열에 위치
                            temp_out_target = (temp_in_loc[0], n-2) # out을 5열에 위치시키되, in과 같은 행에 배치한다.
                        else:
                            temp_in_loc = (temp_in_loc[0], n-1)# in을 6열에 위치  
                            temp_out_target = (temp_out_loc[0], n-2)# out을 5열에 위치
                
                    else:
                        temp_out_target = (temp_out_loc[0], n-2)# 5열에 그냥 위치
                else: # 서로 행이 같다면
                    temp_in_loc = (temp_in_loc[0], n-1) # in을 6열에 일치
                    temp_out_target = (temp_out_loc[0], n-2) # out을 5열에 위치 
    
        else: # out이 Subpuzzle 밖에 있을 때(1,2행 풀 때)
            if (temp_in_loc[0] == target_row+2):
                if (temp_in_loc[1] == n-2): # in이 최하단좌측에 위치한다면
                    temp_in_loc = (temp_in_loc[0]-1, temp_in_loc[1]) # in을 한 칸 위로 미뤄주고(out에 걸리지 않게)
                    
                    if temp_out_loc[1]!=n-1: # 마지막 열이 아니라면
                        temp_out_target = (target_row+2, n-2) # out을 최하단좌측에 위치 
                    else: # 마지막 열이라면
                        temp_out_target = (target_row+2, n-1) # 최하단우측에 위치 
                else: # in이 최하단우측에 존재한다면
                    if temp_out_loc[1]==(n-1): # out이 최우측에 존재할 때는 
                        temp_in_loc = (temp_in_loc[0]-1, n-1) # 한 칸 위로 미루고, 
                        temp_out_target = (target_row+2, n-1) # 최하단좌측에 위치 
                    else: # 나머지는
                        temp_out_target = (target_row+2, n-2) # 최하단 좌측에 위치
            else: # in이 최하단에 없을 때는 편하게
                if (temp_out_loc[1]==(n-1)):
                    temp_out_target = (target_row+2, n-1)
                else:
                    temp_out_target = (target_row+2, n-2)

        if target_loc1==(-1,-1): # t1이 out이었을 때 
            target_loc2 = temp_in_loc 
            target_loc1=temp_out_target
        else:
            target_loc1 = temp_in_loc
            target_loc2=temp_out_target
            
    return target_loc1, target_loc2


# 2x3 퍼즐을 풀 때 t1, t2를 최단거리로 위치시키는 함수.
def GetShortestDouble2(state, size,  target_col, t1, t2,loc1,loc2):
    m,n = size # 2x3  
    target_loc1 = (-1, -1)
    target_loc2 = (-1, -1)
    sub_puzzle=state[m-2:, target_col:target_col+3]

    # 이미 퍼즐 안에 t1,t2이 들어와 있다면
    if (t1 in sub_puzzle)and(t2 in sub_puzzle): # 2x3 Puzzle
        
        if 0 in sub_puzzle: # 0도 들어와있으면 끝
            target_loc1 = loc1
            target_loc2 = loc2
        else: # 0이 없다면, 0이 들어갈 자리를 내주어야 한다. 
            if (loc1[1]!=loc2[1]):
                target_loc1=loc1
                target_loc2=loc2
            elif (loc1[1]==loc2[1]) and (loc1[1]==target_col+2): # 길을 막고 있다면
                if loc1[0]<loc2[0]: # 위에 있는 친구를 한칸 더 미뤄준다.
                    target_loc1 = (loc1[0], target_col+1)
                    target_loc2 = loc2
                else:
                    target_loc2 = (loc2[0], target_col+1)
                    target_loc1 = loc1
            else: # 길을 안 막고 있을 때
                target_loc1 = loc1
                target_loc2 = loc2

    # 두 숫자가 모두 안 들어와있을 때
    elif not ( (t1 in sub_puzzle) or (t2 in sub_puzzle) ):
        # 같은 행에 있으면
        if loc1[0]==loc2[0]:
            if loc1[1]<loc2[1] : # 그대로 평행이동
                target_loc1 = (loc1[0], target_col+1)
                target_loc2 = (loc2[0], target_col+2)
            else:
                target_loc2 = (loc2[0], target_col+1)
                target_loc1 = (loc1[0], target_col+2)
        else: # 다른 행에 있으면
            temp_blank_loc = GetLocation(state, 0) #(공백이 왼쪽에 있을 때랑, 오른 쪽에 있을 때랑 다르
            if loc1[1]==loc2[1]: # 같은 열에 있으면  다
                if loc1[0]<loc2[0]:  # 위에 있는 친구부터 옮긴다. 
                    if temp_blank_loc[1]<loc1[1]: # 만일 공백이 왼쪽에 있다면, 
                        target_loc1 = (loc1[0], target_col+1)
                        target_loc2 = (loc1[0], target_col+2)  # 행 변환이 일어난다.
                    elif(temp_blank_loc[1]>loc1[1]): # 공백이 오른쪽에 있다면, 
                        target_loc1= (loc1[0], target_col+1)
                        target_loc2= (loc2[0], target_col+2) # 행 변환이 일어나지 않는다(뒤로 간).
                    else:
                        raise NameError("is possible?")

                else :
                    if temp_blank_loc[1]<loc2[1]:
                        target_loc2 = (loc2[0], target_col+1)
                        target_loc1 = (loc2[0], target_col+2)  # 단, 이 때 행 변환이 일어난다.
                    elif(temp_blank_loc[1]>loc2[1]): 
                        target_loc2= (loc2[0], target_col+1)
                        target_loc1= (loc1[0], target_col+2) # 행 변환이 일어나지 않는다.
            
            else: # 다른 열에 있으면 
        
                if loc1[1]<loc2[1]: # 왼쪽에 있는 친구부터 옮긴다.

                    if temp_blank_loc[1]>loc1[1]: # 공백이 오른 쪽에 있을 때 
                        if loc1[1]==(loc2[1]-1): # t2의 행 바뀐다(단, t2 안 건들게 우회시켜줄 때 - fbd_loc2 in GetBlankPath + 공백이 더 오른 쪽에 있을 때 
                            target_loc1 = (loc1[0], target_col+1) 
                            target_loc2 = (loc1[0], target_col+2)
                        else:
                            target_loc1 = (loc1[0], target_col+1)
                            target_loc2 = (loc2[0], target_col+2)    
                    else: # 공백이 왼 쪽에 있거나 같을 때 
                        target_loc1 = (loc1[0], target_col+1)
                        target_loc2 = (loc2[0], target_col+2) 


                else:

                    if temp_blank_loc[1]>loc2[1]:
                        if loc2[1]==(loc1[1]-1): # t1의 행 바뀐다. 
                            target_loc2 = (loc2[0], target_col+1) 
                            target_loc1 = (loc2[0], target_col+2)
                        else:
                            target_loc2 = (loc2[0], target_col+1)
                            target_loc1 = (loc1[0], target_col+2)
                    else:
                        target_loc2 = (loc2[0], target_col+1)
                        target_loc1 = (loc1[0], target_col+2)
                    
    else: # 두 숫자 중 하나만 퍼즐에 들어있을 때
        if t1 in sub_puzzle:
            target_loc1=loc1
            temp_in_loc = loc1
            temp_out_loc = loc2
            
        else:
            target_loc2=loc2
            temp_in_loc = loc2
            temp_out_loc = loc1
            
        if temp_in_loc[1]<=(target_col+1): # 좌측 2개 열에 들어있는 경우 
            temp_out_target = (temp_out_loc[0], target_col+2) 
        else: # 3번째 열에 들어가 있는 경우
            
            temp_in_loc = (temp_in_loc[0], target_col+1) # in을 한 칸 미뤄주고, 
            temp_blank_loc = GetLocation(state, 0)

            if temp_in_loc[1]<temp_blank_loc[1]:
                if (temp_in_loc[0]!=temp_out_loc[0]) and (temp_in_loc[1]+1==(temp_out_loc[1]-1)): # 역전현상 일어나는 경우
                    temp_out_target = (temp_in_loc[0], target_col+2)

                else: 
                    temp_out_target = (temp_out_loc[0], target_col+2) # out을 끌고온다.
            else: # 공백이 out을 건드릴 일이 없다면
                temp_out_target = (temp_out_loc[0] ,target_col+2)
                
        # target 할당
        if target_loc1==(-1,-1):
            target_loc1 = temp_out_target
            target_loc2 = temp_in_loc
        else:
            target_loc2 = temp_out_target
            target_loc1 = temp_in_loc

    return target_loc1, target_loc2



# 3x2 퍼즐용 Blank Tile이 최단거리로 들어갈 위치 찾기.
def CheckBlankPath(state, size, blank_loc, target_row, loc1, loc2):
    m,n = size
    sub_puzzle = state[target_row: target_row+3, n-2:]
    # blank도 들어가 있으면 끝
    if 0 in sub_puzzle:
        return []
    
    # 일단 t1,t2이 Subpuzzle의 마지막 2행에 위치할 때(일반적인 경우)
    if (target_row<loc1[0]<=(target_row+2)) and (target_row<loc2[0]<=(target_row+2)):
        if(loc1[0]==loc2[0]): # t1, t2이 같은 행에 있다면
            if blank_loc[0]==loc1[0]: # 공백도 같은 행에 있다면 
                if blank_loc[0]==(target_row+1):
                    if blank_loc[1]==n-3:
                        return ["down", "right"]
                    else:
                        temp_gap = (n-2) - blank_loc[1]
                        return ["down"] + ["right"]*temp_gap
                elif (blank_loc[0]==(target_row+2)): 
                    return ["up", "right"]                    
            
            else: # 공백과 숫자들이 다른 행에 있다면
                if blank_loc[0]<=(target_row+2): # Subpuzzle과 같은 높이에 있을 때 
                    temp_gap = (n-2) - blank_loc[1]
                    return ["right"]*temp_gap # n-1열 까지만 위치시키면 된다. (어차피 공백은 항상 4열쯤에 있다 - 가 너무 안일한 생각이었다. 경우의수가 10^18개인데..)
                elif blank_loc[0]==(target_row+3): # 다른 높이에 있을 시 돌아간다.
                    
                    if blank_loc[1]==(n-1):
                        return ["left", "left" ,"up", "up", "right"] # Check ([[*, *], [*, *], [t1, t2])이고, 공백이 t2 아래에 있을 때 
                    else: # 위에서, 공백이 t1 아래에 있을 때 
                        return ["left", "up", "up", "right"]
                
                else:
                    return

        else: # t1, t2이 다른 행에 있다
            if (loc1[0]>loc2[0]): # t1이 더 아래에 있다면
                temp_low_loc=loc1
                temp_high_loc=loc2
            else:
                temp_low_loc=loc2
                temp_high_loc=loc1
            
            # loc1이 낮은 숫자이다. loc1 대신 low_loc, lo2대신 high_loc
            if temp_low_loc[1]==(n-2): # low : 5열 
                if temp_high_loc[1] == (n-2): # low : 5열, high : 5열 
                    return GetBlankPath(size, blank_loc, temp_low_loc, "right", temp_low_loc[0], fbd_loc=temp_high_loc)
                else: # low : 5열, high : 6열 
                    if blank_loc[0]==(temp_low_loc[0]+1): # 공백이 t1 아래에 있다면 
                        return GetBlankPath(size, blank_loc, temp_low_loc, "right", temp_low_loc[0])
                    elif blank_loc[1]==(temp_low_loc[1]-1): # 공백이 t1 왼쪽에 있다면? t1의 위로. 
                        return GetBlankPath(size, blank_loc, temp_low_loc, "up", temp_low_loc[0]-1) # 위로 길을 열어준다.
                        
            elif temp_low_loc[1] ==(n-1): # t1이 6열에 있다면?
                if blank_loc[0]==(temp_low_loc[0]+1): # 공백이 t1 아래에 있다면
                    return GetBlankPath(size, blank_loc, temp_low_loc, "left", temp_low_loc[0])
                elif blank_loc[1]==(temp_high_loc[1]-1): # 공백이 t2 왼쪽에 있다면
                    return ["down", "right"]
                elif (temp_high_loc[1]==(n-1)): # t2도 6열에 있다면? 
                    if blank_loc[0]==temp_high_loc[0]:
                        return GetBlankPath(size, blank_loc, temp_high_loc, "left", temp_high_loc[0])

                    elif blank_loc[0]==temp_low_loc[0]:   # 이런 건 나올 일 거의 없을 것 같은데, 혹시 모르니 해주자. 
                        return GetBlankPath(size, blank_loc, temp_low_loc, "left", temp_low_loc[0])
                    
                    else:
                        raise NameError("CheckBlankPath 확인")

    elif ((blank_loc[0]-1, blank_loc[1])==loc1) or ((blank_loc[0]-1, blank_loc[1])==loc2) : # 위쪽에 누가 있을 때 (Up tile)
        if (blank_loc[1]==n-2):
            return ["right", "up"]
        else:  # blank_loc[1]==n-1
            return ["left", "up"]

    else: # 그 외 Sub puzzle의 왼쪽 공간에 blank_tile이 위치할 때  
        temp_gap = (n-2) - blank_loc[1]
        temp_blank_loc = (blank_loc[0], blank_loc[1]+temp_gap)
        if (temp_blank_loc==loc1) or (temp_blank_loc==loc2):
            if blank_loc[0]== (target_row+1):
                return ["down"] + ["right"]*temp_gap
            else:
                return ["up"] + ["right"]*temp_gap

        else:
            return ["right"]*temp_gap





# 2x3 퍼즐용 Blank Tiile 찾기.
def CheckBlankPath2(state, size, blank_loc, target_col, loc1, loc2):
    m,n = size
    sub_puzzle = state[m-2:, target_col:target_col+3]
    # blank도 들어가 있으면 끝
    if 0 in sub_puzzle:
        return []
    
    target_loc_col = target_col+2
    gap_to_target = blank_loc[1]-target_loc_col #왼쪽으로 gap_to_target만큼 가야한다. 
    if blank_loc[0]==(m-2): # 공백이 4행에 있고
        # 같은 행에 누군가가 가로막고 있다면
        if ((blank_loc[0], target_loc_col) == loc1) or ((blank_loc[0], target_loc_col) == loc2):
            return (["down"] + ["left"]*gap_to_target)
        else: # 안 가로막고 있다면
            return ["left"]*gap_to_target 
    else: # 공백이 5행에 있고
        if ((blank_loc[0], target_loc_col) == loc1) or ((blank_loc[0], target_loc_col) == loc2):
            return (["up"] + ["left"]*gap_to_target)
        else: # 안 가로막고 있다면
            return ["left"]*gap_to_target 


        




    








In [29]:
def MakeSTT():

    target_state = np.array([[1,2], [3,4], [5,0]])
    target_state2 = np.array([[1,2,3], [4,5,0]])
    STT,hash_array = BBFS(target_state, size=(3,2)) # 3x2 STT
    STT2, hash_array2= BBFS(target_state2, size=(2,3)) # 2x3 STT

    return STT, hash_array, STT2, hash_array2

def SelectMovingOrder_3x2(state, size, blank_loc, t1, t2, loc1, loc2, row, target_loc1, target_loc2, ):
    m,n=size
    if (row<m-3):
        if (loc1[0]<loc2[0]):
            blank_loc = DirectMove(state, size, blank_loc, t1, target_loc1, global_row=row, global_col=n-2, forward=False)
            blank_loc = DirectMove(state, size, blank_loc, t2, target_loc2, global_row=row, global_col=n-2, forward=False, fbd_loc = target_loc1)
        elif (loc1[0]>loc2[0]):
            blank_loc = DirectMove(state, size, blank_loc, t2, target_loc2, global_row=row, global_col=n-2, forward=False)
            blank_loc = DirectMove(state, size, blank_loc, t1, target_loc1, global_row=row, global_col=n-2, forward=False, fbd_loc = target_loc2)
        else:  
            if loc1[1]<loc2[1]: # 만일 같다면 오른쪽에 있는 친구부터. 
                blank_loc = DirectMove(state, size, blank_loc, t2, target_loc2, global_row=row, global_col=n-2, forward=False)
                blank_loc = DirectMove(state, size, blank_loc, t1, target_loc1, global_row=row, global_col=n-2, forward=False, fbd_loc = target_loc2)
            else:
                blank_loc = DirectMove(state, size, blank_loc, t1, target_loc1, global_row=row, global_col=n-2, forward=False)
                blank_loc = DirectMove(state, size, blank_loc, t2, target_loc2, global_row=row, global_col=n-2, forward=False, fbd_loc = target_loc1)
    else: # row == m-3일 때(마지막 3x2)는 오른쪽부터 움직이는 게 낫다.
        if (loc1[1]<loc2[1]):
            blank_loc = DirectMove(state, size, blank_loc, t2, target_loc2, global_row=row, global_col=n-2, forward=False)
            blank_loc = DirectMove(state, size, blank_loc, t1, target_loc1, global_row=row, global_col=n-2, forward=False)#, fbd_loc = target_loc2) 
        elif(loc1[1]>loc2[1]):
            blank_loc = DirectMove(state, size, blank_loc, t1, target_loc1, global_row=row, global_col=n-2, forward=False)
            blank_loc = DirectMove(state, size, blank_loc, t2, target_loc2, global_row=row, global_col=n-2, forward=False)#, fbd_loc = target_loc2) 
        else: # 같다면 위쪽부터 이동하는 게 나음.
            if loc1[0]<loc2[0]:
                blank_loc = DirectMove(state, size, blank_loc, t1, target_loc1, global_row=row, global_col=n-2, forward=False)
                blank_loc = DirectMove(state, size, blank_loc, t2, target_loc2, global_row=row, global_col=n-2, forward=False)#, fbd_loc = target_loc2) 
            if loc1[0]>loc2[0]:
                blank_loc = DirectMove(state, size, blank_loc, t2, target_loc2, global_row=row, global_col=n-2, forward=False)
                blank_loc = DirectMove(state, size, blank_loc, t1, target_loc1, global_row=row, global_col=n-2, forward=False)#, fbd_loc = target_loc2) 
    return blank_loc
    
def SelectMovingOrder_2x3(state, size, blank_loc, t1, t2, loc1, loc2, target_loc1, target_loc2):
    if (loc1[1]<loc2[1]):
            blank_loc = DirectMove(state, size, blank_loc, t1, target_loc1, forward=False, fbd_loc2=loc2)
            blank_loc = DirectMove(state, size, blank_loc, t2, target_loc2, forward=False)
    elif (loc1[1]>loc2[1]):
        blank_loc = DirectMove(state, size, blank_loc, t2, target_loc2, forward=False, fbd_loc2=loc1)
        blank_loc = DirectMove(state, size, blank_loc, t1, target_loc1, forward=False)
    else:
        if loc1[0]>loc2[0]: # 위에 있는 친구부터.
            blank_loc = DirectMove(state, size, blank_loc, t2, target_loc2, forward=False, fbd_loc2=loc1)
            blank_loc = DirectMove(state, size, blank_loc, t1, target_loc1, forward=False)
        else:
            blank_loc = DirectMove(state, size, blank_loc, t1, target_loc1,forward=False, fbd_loc2=loc2)
            blank_loc = DirectMove(state, size, blank_loc, t2, target_loc2, forward=False)


    
    return blank_loc


In [31]:


def GetLocation(state, number):

    temp = np.where(state == number)
    
    return (temp[0][0], temp[1][0]) # (y, x) 좌표


def IsSolvable(state, size):
    m, n = size
    bi=(np.where(state==0)[0] + 1) # blanktile의 row (1부터 시작)

    total_inverse = 0   

    for i in range(m):
        for j in range(n):
            inverse_value=get_inverse(state, size, (i,j), solvable=True)

            total_inverse+=inverse_value

    if (n%2) and not (total_inverse%2): # n이 홀수이면, inver는 짝수
        return True
    elif not (n%2) and ((m-bi)%2 ==(total_inverse%2)): # blank tile이 (아래에서부터) 홀수에 위치하면 inverse는 짝수
        return True
    else:
        return False
    

# hash 구할 때
# solvable 판단할 때(solvable=True)
def get_inverse(state, size, loc, solvable=False): 
    # input 
    #   state : 퍼즐 (2d array)
    #   size : 퍼즐 사이즈 (tuple)
    #   loc : inverse 값을 구하려는 타일 좌표 (tuple)(row, col)
    # output
    #   inverse value : 우하단 방향으로, 해당 좌표의 값보다 작은 타일의 개수.

    m, n = size
    y, x = loc
    
    anchor_value = state[y, x] # 타일의 값
    count = 0
    
    for i in range(y, m):
        if (i==y):# 첫 줄
            for j in range(x+1, n):
                if(state[i, j] < anchor_value):
                    if solvable:
                        if state[i,j]!=0: # solvable 구할 때는 blanktile 포함 x
                            count+=1
                    else:
                        count +=1

        else: # 나머지 줄 
            for j in range(0, n):
                if(state[i, j] < anchor_value):
                    if solvable:
                        if state[i,j]!=0:
                            count+=1
                    else:
                        count+=1

    return count

# 3x2 Sub-puzzle에서, 6!/2 = 360가지의 puzzle -> 360개의 hash 
def hash(state):

    hash_value = 1
    i = 0
    m, n = np.shape(state)
    for row in range(m-1, -1, -1):
        for col in range(n-1, -1, -1):
            temp_inverse = get_inverse(state, (m, n), (row, col), solvable=False)

            hash_value += (temp_inverse * fact(i))
            i+=1 # 좌표 
    
    return hash_value

# 대충 만든 swap 함수 (어차피 in-place Algorithm이라 굳이 다른 자료구조 안 써도 괜찮)
def move(state, size, blank_loc, direct):
    global move_count
    m, n = size
    y, x = blank_loc
    move_count+=1
    if direct=="left":
        if x-1<0:
            return
        else:
            state[y, x], state[y, x-1] = state[y, x-1], state[y, x] # swap 
            return (y, x-1)
    elif direct=="right":
        if x+1>=n:
            return
        else:
            state[y, x], state[y, x+1] = state[y, x+1], state[y, x] # swap
            return (y, x+1)

    elif direct=="up":
        if y-1<0:
            return
        else:   
            state[y, x], state[y-1, x] = state[y-1, x], state[y, x] # swap
            return (y-1, x)
    elif direct=="down":
        if y+1>=m:
            return
        else:
            state[y, x], state[y+1, x] = state[y+1, x], state[y, x] # swap
            return (y+1, x)

    else:
        raise NameError("impossible")   


# SST table 만들기용 move 함수 (C++에선 Struct)
def move_sst(state2, size, blank_loc, direct):
    state = state2.copy() # deepcopy

    m, n = size
    y, x = blank_loc
    if direct=="left":
        if x-1<0:
            return
        else:
            state[y, x], state[y, x-1] = state[y, x-1], state[y, x] # swap 
            return state, (y, x-1)
    elif direct=="right":
        if x+1>=n:
            return
        else:
            state[y, x], state[y, x+1] = state[y, x+1], state[y, x] # swap
            return state, (y, x+1)

    elif direct=="up":
        if y-1<0:
            return
        else:   
            state[y, x], state[y-1, x] = state[y-1, x], state[y, x] # swap
            return state, (y-1, x)
    elif direct=="down":
        if y+1>=m:
            return
        else:
            state[y, x], state[y+1, x] = state[y+1, x], state[y, x] # swap
            return state, (y+1, x)

    else:
        raise NameError("제대로 옮기세요") 



